In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pyautogui
from bs4 import BeautifulSoup
import requests

import pandas as pd
import time

import pickle
from tqdm import tqdm

### 상품 URL 추출

In [289]:
driver = webdriver.Chrome()
url = 'https://shopping.naver.com/beauty/category?menu=10003291'
driver.get(url)

array = []

SCROLL_PAUSE_SEC = 3
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_SEC)
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    SCROLL_PAUSE_SEC = 5
    tmp = driver.find_elements(By.CSS_SELECTOR, "._2KkiZqJBZF > a")
#     array.append(tmp[0].get_attribute('href'))
#     print(len(array))
    
    for i in range(len(tmp)):
        array.append(tmp[i].get_attribute('href'))

    if new_height == last_height:
        try:
            driver.find_element_by_css_selector(".mye4qd").click()
        except:
            break
    last_height = new_height
    
    if len(array) >= 5000:
        break

In [290]:
len(array)

5000

In [291]:
with open('naver_url.pickle', 'wb') as f:
    pickle.dump(array, f)

In [6]:
with open('naver_url.pickle', 'rb') as f:
    array = pickle.load(f)

In [7]:
array

['https://shopping.naver.com/beauty/stores/1000009464/products/7263586848',
 'https://shopping.naver.com/beauty/stores/1000009464/products/5036447034',
 'https://shopping.naver.com/beauty/stores/1000009464/products/7263590874',
 'https://shopping.naver.com/beauty/stores/1000019604/products/6400365010',
 'https://shopping.naver.com/beauty/stores/1000019382/products/2363490746',
 'https://shopping.naver.com/beauty/stores/100002853/products/6771814380',
 'https://shopping.naver.com/beauty/stores/100032918/products/2303064285',
 'https://shopping.naver.com/beauty/stores/100002853/products/5945562605',
 'https://shopping.naver.com/beauty/stores/1000009763/products/7243224860',
 'https://shopping.naver.com/beauty/stores/1000012777/products/1001438041',
 'https://shopping.naver.com/beauty/stores/1000009464/products/6341865999',
 'https://shopping.naver.com/beauty/stores/101672368/products/7239596884',
 'https://shopping.naver.com/beauty/stores/1000014372/products/4312275658',
 'https://shoppi

### 각 상품 정보 추출

#### 1번 상품

In [298]:
# 1번 상품
url = f"{array[0]}"
response = requests.get(url)
dom = BeautifulSoup(response.text, 'html.parser')

# 상품명
name = dom.select("._3oDjSvLwq9")[0].text
# 가격
price = dom.find("span",{"class":"_1LY7DqCnwR"}).text

# 데이터 프레임 컬럼
find_columns = dom.select("._1Hbih69XFT")[0]
columns = find_columns.find_all("th",{"class":"_15qeGNn6Dt"})
df_columns = ["상품명", "가격"]
for i in range(len(columns)):
    df_columns.append(columns[i].text)

# 데이터 프레임 로우
find_rows = dom.select("._1Hbih69XFT")[0]
rows = find_rows.find_all("td",{"class":"jvlKiI0U_y"})
df_rows = [name, price]
for i in range(len(rows)):
    df_rows.append(rows[i].text)

# 데이터프레임 형성
df = pd.DataFrame(columns = df_columns)
df.loc[0] = df_rows

#### 2번부터

In [299]:
for i in tqdm(array[1:]):
    url = f"{i}"
    response = requests.get(url)
    dom = BeautifulSoup(response.text, 'html.parser')
    
    # 상품명
    name = dom.select("._3oDjSvLwq9")[0].text
    # 가격
    price = dom.find("span",{"class":"_1LY7DqCnwR"}).text
    
    # 데이터 프레임 컬럼
    find_columns = dom.select("._1Hbih69XFT")[0]
    columns = find_columns.find_all("th",{"class":"_15qeGNn6Dt"})
    df_columns = ["상품명", "가격"]
    for i in range(len(columns)):
        df_columns.append(columns[i].text)

    # 데이터 프레임 로우
    find_rows = dom.select("._1Hbih69XFT")[0]
    rows = find_rows.find_all("td",{"class":"jvlKiI0U_y"})
    df_rows = [name, price]
    for i in range(len(rows)):
        df_rows.append(rows[i].text)

    # 데이터프레임 형성
    n_df = pd.DataFrame(columns = df_columns)
    n_df.loc[0] = df_rows
    
    # 데이터프레임 병합
    # 기존에 만들어진 df, n_df = n_df
    df = pd.concat([df, n_df], axis=0)

100%|████████████████████████████████████████████████████████████████████████████| 4999/4999 [1:00:36<00:00,  1.37it/s]


In [300]:
df.isnull().sum()

상품명                0
가격                 0
상품번호               0
상품상태               0
브랜드              190
원산지                0
세부제품특징           732
사용부위            1776
사용시간            2987
제품형태            2660
용량              2511
용기형태            3903
피부타입            1718
주요제품특징            36
모델명             1636
제조사              725
혜택조건            4371
세트수량            4854
반품안심케어 가입 안내    4921
이벤트             3952
제조일자            4598
유효일자            4524
기타              4935
사은품             4599
PA지수            4666
자외선차단지수         4660
성분              4970
종류              4956
세트구성            4996
매수              4952
구분              4974
dtype: int64

### 데이터프레임 저장

In [302]:
df.to_csv("naver.csv")

In [11]:
pd.read_csv('naver.csv').head()

,Unnamed: 0,상품명,가격,상품번호,상품상태,브랜드,원산지,세부제품특징,사용부위,사용시간,...,유효일자,기타,사은품,PA지수,자외선차단지수,성분,종류,세트구성,매수,구분
0,0,[스킨푸드][당근2종키트증정] 캐롯 카로틴 카밍 워터 패드 30매 135g_당근패드,"13,000",7263586848,신상품,스킨푸드,대한민국,"피부진정, 피부결정돈, 상쾌함",페이스용,"낮, 밤",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,[스킨푸드][당근2종키트증정] 캐롯 카로틴 카밍 워터 패드 60매_당근패드,"26,000",5036447034,신상품,스킨푸드,대한민국,"피부진정, 피부결정돈, 상쾌함",페이스용,"낮, 밤",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,[스킨푸드][미나리2종키트증정] 판토테닉 워터 파슬리 패드 30매 135g_미나리패드,"13,000",7263590874,신상품,스킨푸드(33493),대한민국,"각질케어, 흡수력",페이스용,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,[1+1]바버 멀티 비타민 앰플 (2mlx7병)- 바버공식수입사,"60,000",6400365010,신상품,BABOR,독일산((주)이넬화장품),"저자극, 피부강화, 각질케어, 피부탄력, 영양공급",페이스용,"낮, 밤",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,데저트에센스 100% 냉압착 비정제 퓨어 호호바오일 118ml,"27,700",2363490746,신상품,데저트에센스,미국산(컨트리 라이프 엘엘씨.),"저자극, 피부보호",페이스용,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
